In [1]:

from darts.utils.timeseries_generation import gaussian_timeseries, linear_timeseries, sine_timeseries
from darts.models import TransformerModel
from darts.metrics import mape, smape

from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries

import pandas as pd
from darts import TimeSeries

/home/chef-haider/miniconda3/envs/myenv/lib/python3.8/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


ImportError: cannot import name 'croston_classic' from 'statsforecast.models' (/home/chef-haider/miniconda3/envs/myenv/lib/python3.8/site-packages/statsforecast/models.py)

In [ ]:
df = pd.read_csv('/home/chef-haider/Downloads/btc-sent-f4.csv')

df.time = pd.to_datetime(df.time)
df = df.set_index('time')
df = df.sort_index()
df = df.astype('float')

df['timestamp'] = df.index.tz_localize(None)

In [ ]:
model = TransformerModel.load('/home/chef-haider/Downloads/model1.pth.tar')

AttributeError: Can't get attribute '_FaultTolerantMode' on <module 'pytorch_lightning.utilities.enums' from '/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/enums.py'>

In [ ]:
series_open = TimeSeries.from_dataframe(df, 'timestamp', 'open')

open_scaled = Scaler()
series_open_scaled = open_scaled.fit_transform(series_open)

#open_train, open_val = series_open_scaled[:-32], series_open_scaled[-32:]

In [ ]:
pred = model.predict(n=12, series=series_open_scaled)
pred = series_open_scaled[-1].append(pred)


df_res = pd.DataFrame()
df_res.index = pred.pd_dataframe().index .append( series_open_scaled[-36:].pd_dataframe().index )

df_res['previous'] = open_scaled.inverse_transform(series_open_scaled[-36-24:]).pd_series()
df_res['predicted'] =  open_scaled.inverse_transform(pred).pd_series()

df_res.to_csv('output12.csv')

Predicting: 0it [00:00, ?it/s]

In [ ]:
print('MAPE = {:.2f}%'.format(mape(pred,open_val)))

MAPE = 45.25%


In [ ]:
pred = model.predict(n=12, series=open_train)
pred = open_train[-1].append(pred)


df_res = pd.DataFrame()
df_res.index = pred.pd_dataframe().index .append( open_train[-36:].pd_dataframe().index )

df_res['previous'] = open_scaled.inverse_transform(open_train[-36-24:]).pd_series()
df_res['actual'] =  open_scaled.inverse_transform((open_train[-1].append(open_val[:12]))).pd_series()
df_res['predicted'] =  open_scaled.inverse_transform(pred).pd_series()

df_res.to_csv('outputx.csv')

Predicting: 0it [00:00, ?it/s]

In [ ]:
)

In [ ]:
df_res['predicted'].iloc[-1]

41363.25

In [ ]:
import plotly.express as px

df_res = pd.read_csv('outputx.csv')
df_res.timestamp = pd.to_datetime(df_res['timestamp'])

fig = px.line(df_res, x="timestamp", y=['actual','predicted','previous'],
              title='LSTM Multivariate with sentiment scores')

fig.show()